# Etape 5 : Challenge TETLNET

Telnet utilise le protocole IP, et utilise le port 23.

L'objectif est :

- Retrouver le login/mot de passe d'une connexion telnet dans le fichier de capture proposé
- Ecrire un script Python permettant de récupérer automatiquement les login/ mot de passe dans cette   capture
- Ecrire un script Python permettant de reconnaître une connexion telnet et de récupérer automatiquement les login/mot de passe échangés sur le réseau 
A vrai dire la seule difficulté supplémentaire consiste à repérer comment est transmis le mot de passe, ce qui constitue la seule réelle nouveauté.

#### Test d'affichage partie des lignes concerné :

In [3]:
from scapy.all import *

listetrame=rdpcap("telnet-total.pcapng") #ici on importe notre trame

listetrame.summary()

Ether / ARP who has 10.2.4.4 says 10.2.4.1 / Padding
Ether / ARP who has 10.2.4.4 says 10.2.4.1 / Padding
Ether / IPv6 / UDP / DNS Qry "b'28.122.35.161.in-addr.arpa.'" 
Ether / IP / UDP / DNS Qry "b'28.122.35.161.in-addr.arpa.'" 
Ether / IPv6 / ICMPv6ND_NS / ICMPv6 Neighbor Discovery Option - Source Link-Layer Address 00:13:1a:a3:e1:18
802.3 b0:aa:77:e0:f8:8d > 01:80:c2:00:00:00 / LLC / STP / Padding
Ether / IPv6 / ICMPv6ND_NS / ICMPv6 Neighbor Discovery Option - Source Link-Layer Address 00:13:1a:a3:e1:18
Ether / ARP who has 10.2.4.4 says 10.2.4.1 / Padding
Ether / ARP who has 10.2.4.4 says 10.2.4.1 / Padding
Ether / IPv6 / ICMPv6ND_NS / ICMPv6 Neighbor Discovery Option - Source Link-Layer Address 00:13:1a:a3:e1:18
Ether / IPv6 / UDP 2a00:1450:4006:800::200a:https > 2001:660:6701:30cc:b0c9:8f6f:ed17:90a2:54430 / Raw
Ether / IPv6 / UDP 2a00:1450:4006:800::200a:https > 2001:660:6701:30cc:b0c9:8f6f:ed17:90a2:54430 / Raw
Ether / IPv6 / UDP 2001:660:6701:30cc:b0c9:8f6f:ed17:90a2:54430 > 2a

In [6]:
listetrame[83][Raw] #la trame avec le login trouver dans Wireshark

<Raw  load='734cc26aaf98 login: ' |>

Après avoir visualiser sur wireshard on vois que le login est "marie"

In [7]:
listetrame[118][Raw] #la trame avec le password trouver dans Wireshark

<Raw  load='Password: ' |>

Après avoir visualiser sur wireshark on vois que le mdp est : poppins

In [8]:
listetrame[282][Raw]

<Raw  load='ceci est un fichier indispensable ...\r\n' |>

In [9]:
listetrame[284][Raw]

<Raw  load='\x1b]0;marie@734cc26aaf98: ~\x07\x1bmarie@734cc26aaf98\x1b:\x1b~\x1b$ ' |>

# Programme final : 

In [15]:
from scapy.all import *

listetrame=rdpcap("telnet-total.pcapng") #ici on importe notre trame

login=[] #pour stocker tous les caratère du login envoyer 1 par 1 en telnet
psw=[] #pour stocker tous les caratère du mots de passe envoyer 1 par 1 en telnet

for elm in listetrame : #pour tout les element dans la trame
    try :
        if elm['Ether'].type==0x0800: #si l'en tête etherne a pour type suivant l'IPv4
            if elm['TCP'].dport == 23 or elm['TCP'].sport == 23: #si le fort source ou destination correspont à 23 donc telnet
                if elm['Raw'].load != '': # Vérifie si les données ne sont pas égale à une chaîne d'octets vide (''). 
                    trametrier=elm['Raw'].load #on associe les donnée a la variable trame trier
                    print(trametrier)
    
    
    except:
        continue 

b'\xff\xfd\x03\xff\xfb\x18\xff\xfb\x1f\xff\xfb \xff\xfb!\xff\xfb"\xff\xfb\'\xff\xfd\x05\xff\xfb#'
b"\xff\xfd\x18\xff\xfd \xff\xfd#\xff\xfd'"
b'\xff\xfb\x03\xff\xfd\x1f\xff\xfd!\xff\xfe"\xff\xfb\x05\xff\xfa \x01\xff\xf0\xff\xfa#\x01\xff\xf0\xff\xfa\'\x01\xff\xf0\xff\xfa\x18\x01\xff\xf0'
b"\xff\xfa\x1f\x00P\x00\x18\xff\xf0\xff\xfa \x0038400,38400\xff\xf0\xff\xfa#\x00JJ-BURO:1\xff\xf0\xff\xfa'\x00\x00DISPLAY\x01JJ-BURO:1\xff\xf0\xff\xfa\x18\x00xterm-256color\xff\xf0"
b'\xff\xfd\x01'
b'\xff\xfc\x01'
b'\xff\xfb\x01'
b'\xff\xfd\x01'
b'Ubuntu 16.04.6 LTS\r\n'
b'734cc26aaf98 login: '
b'm'
b'm'
b'a'
b'a'
b'r'
b'r'
b'i'
b'i'
b'e'
b'e'
b'\r\x00'
b'\r\n'
b'Password: '
b'p'
b'o'
b'p'
b'p'
b'i'
b'n'
b's'
b'\r\x00'
b'\r\n'
b'Last login: Wed Mar 23 12:17:37 UTC 2022 from rt-jjb-003.univ-pau.fr on pts/0\r\n'
b'Welcome to Ubuntu 16.04.6 LTS (GNU/Linux 4.15.0-163-generic x86_64)\r\n\r\n * Documentation:  https://help.ubuntu.com\r\n * Management:     https://landscape.canonical.com\r\n * Support:        h

Vérifie si la charge utile brute n'est pas égale à une chaîne d'octets vide (b''). 